In [ ]:
#@title Settings
use_ngrok = False #@param {type:"boolean"}
ngrok_auth_key = "" #@param {type:"string"}
branch = 'master' #@param ["master", "dev"]
use_flash_attention = True #@param {type:"boolean"}
save_models_to_gdrive = False #@param {type:"boolean"}
save_images_to_gdrive = False #@param {type:"boolean"}


In [ ]:
if save_models_to_gdrive or save_images_to_gdrive:
  from google.colab import drive

  drive.mount('/content/gdrive')

In [ ]:
!python --version

[gpu] = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print (gpu)

import torch
print(torch.__version__)

!nvidia-smi

In [ ]:
%cd /content

!git clone -b {branch} https://github.com/seruva19/kubin.git
%cd /content/kubin

!pip install -r requirements.txt
if use_flash_attention:
  !pip install https://github.com/seruva19/flash-attn-wheels/raw/main/torch2.0.0%2Bcu118/flash_attn-1.0.1-cp310-cp310-linux_x86_64.whl

In [ ]:
if use_ngrok: 
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_auth_key)

  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

  print(ngrok.connect(7860, "http"))

In [ ]:
!python src/kubin.py \
  --cache-dir={"/content/gdrive/MyDrive/kubin/models" if save_models_to_gdrive else "/content/models"} \
  --output-dir={"/content/gdrive/MyDrive/kubin/output" if save_images_to_gdrive else "/content/output"} \
  --share={"none" if use_ngrok else "gradio"} \
  {"--use-flash-attention" if use_flash_attention else ""}